In [1]:
import string 
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', 100) 
pd.set_option('display.max_rows', 10) 
pd.set_option('display.width', 1000) 
from itertools import cycle 

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import sklearn
import datetime
####
# scikit-learn version is 0.24.1.
# pandas version is 1.2.3.
# numpy version is 1.20.1.

In [4]:
df=pd.read_csv(r'D:\учеба\Kaggle\Биржа\US1.AAPL_210501_210901.csv' ,sep=',')

In [5]:
#фичи delta1 - разница <CLOSE> с предыдущим временным промежутком,
# delta2 - с точкой двумя промежутками раньше
delta1=[0 for i in range(0,df["<TICKER>"].size)]
delta2=[0 for i in range(0,df["<TICKER>"].size)]
i=1
while (i<df["<TICKER>"].size):
    delta1[i]=df["<CLOSE>"][i]-df["<CLOSE>"][i-1]
    i+=1
df['delta1']=delta1
delta1=[0 for i in range(0,df["<TICKER>"].size)]

i=2
while (i<df["<TICKER>"].size):
    delta2[i]=df["<CLOSE>"][i]-df["<CLOSE>"][i-2]
    i+=1
df['delta2']=delta2
df

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,delta1,delta2
0,US1.AAPL,1,03/05/21,16:31,132.03,132.11,131.830,132.070,52332,0.000,0.000
1,US1.AAPL,1,03/05/21,16:32,132.06,132.28,132.000,132.190,33828,0.120,0.000
2,US1.AAPL,1,03/05/21,16:33,132.20,132.41,132.175,132.330,19659,0.140,0.260
3,US1.AAPL,1,03/05/21,16:34,132.34,132.65,132.225,132.630,35281,0.300,0.440
4,US1.AAPL,1,03/05/21,16:35,132.63,132.74,132.570,132.660,22357,0.030,0.330
...,...,...,...,...,...,...,...,...,...,...,...
33143,US1.AAPL,1,31/08/21,22:56,151.67,151.84,151.650,151.765,74004,0.105,-0.145
33144,US1.AAPL,1,31/08/21,22:57,151.76,151.90,151.760,151.880,76975,0.115,0.220
33145,US1.AAPL,1,31/08/21,22:58,151.88,151.93,151.820,151.930,56229,0.050,0.165
33146,US1.AAPL,1,31/08/21,22:59,151.93,151.94,151.810,151.840,60126,-0.090,-0.040


In [6]:
#добавляю фичу "день недели" + форматирую дату с дд\мм\гг на дд\мм\гггг
day=[0 for i in range(0,df["<TICKER>"].size)]
from datetime import datetime, date, time
i=0
while (i<df["<TICKER>"].size):
    df["<DATE>"][i]=df["<DATE>"][i][0:6]+"20"+df["<DATE>"][i][6:8]
    day[i]=datetime.weekday(datetime.strptime(df["<DATE>"][i]+" "+df["<TIME>"][i], 
                             "%d/%m/%Y %H:%M"))
    i+=1
df['day']=day

<ipython-input-6-314177c4a164>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["<DATE>"][i]=df["<DATE>"][i][0:6]+"20"+df["<DATE>"][i][6:8]


In [7]:
#добавляю фичу "время" - в секундах с начала торгов (16:30)
size=len(df["<TICKER>"])
hours=[0 for i in range(size)]
mins=[0 for i in range(size)]
secs=[0 for i in range(size)]
for i in range(size):
    hours[i]=int(df["<TIME>"][i][0:2])
    mins[i]=int(df["<TIME>"][i][3:5])
TIME=[0 for i in range(0,df["<TICKER>"].size)]
for i in range(size):
    TIME[i]=(hours[i]-16)*3600+mins[i]*60-1860
    
df['time']=TIME

In [8]:
#метки для обучения (значения <CLOSE> на следующем временном промежутке)
labels=[float(0.0) for i in range(0,df["<TICKER>"].size)]
labels=np.float32(labels)
df['label']=labels
for i in range(size-1):
    df['label'][i]=(df['<CLOSE>'][i+1])

<ipython-input-8-99c929464daa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i]=(df['<CLOSE>'][i+1])


In [9]:
#формирование обучающей, тестовой выборок
train={
    'time':df['time'][0:size-1],
    '<CLOSE>':df['<CLOSE>'][0:size-1],
      'delta1':df['delta1'][0:size-1],
    'delta2':df['delta2'][0:size-1],
'day':df['day'][0:size-5]}
df_train=pd.DataFrame(data=train)
X = df_train

y=df['label'][0:size-1]

from sklearn.model_selection import train_test_split

In [10]:
y=df['label'][0:size-1]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state =0)


In [12]:
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

In [13]:
y_test=y_test.reset_index(drop=True)

In [14]:
# обучение
count=5
mean_square_err=[0]*count
deviation=[0]*count
for depth in range(count):
    regr = RandomForestRegressor(max_depth=5+depth, random_state=0,min_samples_leaf=5,criterion='mse')
    regr.fit(X_train,y_train)
    y_pred=regr.predict(X_test)
    for i in range(y_pred.size):
        mean_square_err[depth]+=(y_test[i]-y_pred[i])**2
    mean_square_err[depth]/=y_pred.size
    mean_square_err[depth]=mean_square_err[depth]**0.5
    for i in range(y_pred.size):
        deviation[depth]+=(y_test[i]-y_pred[i])*100/y_test[i]
    deviation[depth]/=y_pred.size

In [15]:
deviation,mean_square_err

([-0.00036688802947013694,
  -0.00011869886742377729,
  -0.000162788667942917,
  -0.00031346065033381403,
  -0.00024601888090846176],
 [0.22272721567863324,
  0.12722706775664847,
  0.096405032593775,
  0.09203731889350165,
  0.09185098924534714])

In [16]:
# итоговая модель:
model = RandomForestRegressor(max_depth=8, random_state=0,min_samples_leaf=5)
model.fit(X_train,y_train)

RandomForestRegressor(max_depth=8, min_samples_leaf=5, random_state=0)